In [ ]:
#-----------------------------------------------------------------------------
#  Work for MobileNet
#  author @HarshitaSharma
#  Code Collaborator : @Kottana Priyanka
#  Dated : 11 December , 2020
# Kaggle dataset used .
#-----------------------------------------------------------------------------

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
  #  for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install resnet

In [ ]:
import keras
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications import MobileNet
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam
import warnings
warnings.filterwarnings("ignore")
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, classification_report
import seaborn as sns



from keras.applications.inception_v3 import InceptionV3

#from generator import DataGenerator
from keras.callbacks import ModelCheckpoint

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import glob
from itertools import repeat
from itertools import product



from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from collections import Counter

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


import warnings
warnings.filterwarnings("ignore")

print("The TensorFlow version is ", tf.__version__)
print("The Keras version is ", tf.keras.__version__)

In [ ]:
# Data path
TRAIN_PATH = r'../input/kermany2018/OCT2017 /train/'
VAL_PATH = r'../input/kermany2018/OCT2017 /val/'
TEST_PATH = r'../input/kermany2018/OCT2017 /test/'

# Model/path
MODEL_WEIGHTS_PATH = r'../input/oct-resnet-feature-extraction/'
FE_FILE = 'oct_mobilenet_fe_model.h5'
FT_FILE = 'oct_mobilenet_ft_model.h5'


# Parameters
CLASSES = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
NUM_CLASSES = len(CLASSES)
IMG_DIMS = 300  #224
IMG_SHAPE = (IMG_DIMS, IMG_DIMS, 3)

FE_EPOCHS = 7   # feature extraction epochs
FT_EPOCHS = 5   # fine tuning epochs (LR decr.)

BATCH_SIZE = 16
OPTIMIZER = Adam()

# Training flags 
# due to Kaggle training time limits, we set a flag to fine-tune
# for 5 (FT_1) or 5+5 (FT_2) epochs
TRAIN_FT_2 = 0 # set to 0/1 to fine tune for 5 or 5+5 epoch


In [ ]:
# load data, plot class distribution

def load_data(path, label, class_type):
    
    image_list = glob.glob(path + class_type + '/*')
    label_list = list(repeat(label, len(image_list)))
    
    data = list(zip(image_list,label_list))
    df = pd.DataFrame(data, columns = ['images', 'labels'])

    return df
        

def plot_dataset_imbalance(sample_distribution): 
    colors = ['blue', 'red', 'pink', 'turquoise']
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar(CLASSES, sample_distribution, color=colors)
    plt.show()
        
        
        
for dataset, path in list(zip(['train', 'val', 'test'], [TRAIN_PATH, VAL_PATH, TEST_PATH])):
  
    for label,class_type in enumerate(CLASSES):
        df_var = "df_" + dataset + "_" + class_type
        vars()[df_var] = load_data(path, str(label), class_type)
        sample_size_var = "num_" + class_type
        vars()[sample_size_var] = vars()[df_var].shape[0]        
        print('The size of ', df_var, ' is {} '.format(vars()[df_var].shape))
    
    sample_distribution = [num_CNV, num_DME, num_DRUSEN, num_NORMAL]
    plot_dataset_imbalance(sample_distribution)

In [ ]:

fig=plt.figure(figsize=(12, 10))
cols = 3
rows = 4

for idx, rand_idx in enumerate(np.random.randint(0,1000, size=cols*rows)):
    class_type = CLASSES[int(idx/cols)]
    df_var = "df_train_" + class_type
    img = vars()[df_var]['images'][rand_idx]
    img = plt.imread(img)     
    ax = fig.add_subplot(rows, cols, idx+1)
    if idx%cols==0:
        plt.ylabel(CLASSES[int(idx/cols)], fontsize=16)
    plt.imshow(img, cmap='gist_gray')

In [ ]:
def get_train_set(rebalance=0):
    
    if rebalance == 0:
        df_train_upsample = pd.concat([df_train_CNV, 
                               df_train_DME,
                               df_train_DRUSEN,
                               df_train_NORMAL], axis=0).reset_index(drop=True)
    else:
        df_train_upsample = pd.concat([df_train_CNV, 
                               #df_train_DME,
                               #df_train_DRUSEN,
                               df_train_DME, df_train_DME, 
                               df_train_DRUSEN, df_train_DRUSEN, 
                               df_train_NORMAL], axis=0).reset_index(drop=True)

    df_train = shuffle(df_train_upsample)
    print('The size of df_train is {}'.format(df_train.shape))
      
    return df_train


In [ ]:
df_train = get_train_set(rebalance=0)
df_train.head(10)

In [ ]:
def get_test_set():
    
    df_test_combined = pd.concat([df_test_CNV, df_val_CNV, 
                                  df_test_DME, df_val_DME, 
                                  df_test_DRUSEN, df_val_DRUSEN,
                                  df_test_NORMAL, df_val_NORMAL], 
                                 axis=0).reset_index(drop=True)
    df_test = shuffle(df_test_combined)
    print('The size of df_test is {}'.format(df_test.shape))

    return df_test

In [ ]:
df_test = get_test_set()

In [ ]:
def get_train_val_split_data(df_train, test_size=0.05):
# split the train set into a train/val set

    # select the column that we will use for stratification
    y = df_train['labels']
    df_train, df_val = train_test_split(df_train, test_size=test_size, random_state=2020, stratify=y)

    print('The size of df_train is {}'.format(df_train.shape))
    print('The size of df_val is {}'.format(df_val.shape))
    
    return df_train, df_val

In [ ]:
def plot_dataset_distribution(df_train, df_test):
    
    fig, ax = plt.subplots(1,2, figsize=(10, 6))
    colors = ['blue', 'red', 'pink', 'turquoise']


    for index,dataset in enumerate(['train', 'test']):
    
        df_var = "df_" + dataset 
        counts = vars()[df_var]['labels'].value_counts().sort_index()
        ax[index].pie(counts, labels=CLASSES, autopct='%.1f %%', colors=colors)
        ax[index].set_title( '{} set'.format(dataset))
    
    fig.suptitle('Class distribution for train and test datasets',
                 y=1.1, fontsize=16)
    fig.tight_layout()

In [ ]:
df_train, df_val = get_train_val_split_data(df_train)

In [ ]:
#train_counts = df_train['labels'].value_counts().sort_index()
#test_counts = df_test['labels'].value_counts().sort_index()


In [ ]:
plot_dataset_distribution(df_train, df_test) 

In [ ]:
#Get class weights
def get_class_weights(verbose=1):
    
    counter = Counter(df_train.labels)                          
    max_count = float(max(counter.values()))  
    class_weights = {int(class_label) : max_count/num_images for class_label, 
                     num_images in counter.items()}                     
    if verbose:
        print('Class weights: \n', class_weights)
    return class_weights

In [ ]:
class_weights = get_class_weights()

In [ ]:
#Find image size dimensions
def find_img_dims(image_list):
    
    min_size = []
    max_size = []
    
    for i in range(len(image_list)):
        x=image_list[i]
        im = Image.open(x)
        min_size.append(min(im.size))
        max_size.append(max(im.size))
    
    return min(min_size), max(max_size)

In [ ]:
image_list = df_train.images
image_list=list(image_list)
print(len(image_list))
min_size, max_size = find_img_dims(image_list)
print('The min and max image dims are {} and {} respectively.'
      .format(min_size, max_size))

In [ ]:
#We have set the model input image size (IMG_DIMS) to 300x300 pixels which is the expected input resolution for the EfficientNet B3 model. This resolution is smaller than the smallest image in the data set.

In [ ]:
#Image Augmentation/Data Generator


In [ ]:
def get_train_data(augmentation=0):
    
    if augmentation:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    fill_mode='nearest',
                                    zoom_range = 0.3,
                                    horizontal_flip = True)        
    else:    
        train_datagen = ImageDataGenerator(rescale=1./255)
        
    
    train_data= train_datagen.flow_from_dataframe(dataframe=df_train, 
                                            #directory=TRAIN_IMG_DIR, 
                                            directory=None,    # paths specified in x_col
                                            x_col="images", 
                                            y_col="labels", 
                                            class_mode="categorical",  # for multiclass
                                            target_size=(IMG_DIMS, IMG_DIMS),
                                            batch_size=BATCH_SIZE)
    return train_data

In [ ]:
def get_valid_data():
    
    valid_datagen = ImageDataGenerator(rescale=1./255)
    valid_data = valid_datagen.flow_from_dataframe(dataframe=df_val, 
                                             directory=None, 
                                             x_col="images", 
                                             y_col="labels", 
                                             class_mode="categorical",
                                             shuffle= True,
                                             target_size=(IMG_DIMS, IMG_DIMS),
                                             batch_size=BATCH_SIZE)
    return valid_data

In [ ]:
def get_test_data():
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_dataframe(dataframe=df_test, 
                                             directory=None, 
                                             x_col="images", 
                                             y_col="labels", 
                                             class_mode="categorical",
                                             shuffle= False,
                                             target_size=(IMG_DIMS, IMG_DIMS),
                                             batch_size=BATCH_SIZE)
    return test_data

In [ ]:
#Training
#Plot training performance
def display_training_curves(accuracy, loss, start_epoch):
       
    fig, ax = plt.subplots(2,1, figsize=(10, 8))
   

    for index,lcurve in enumerate(['accuracy', 'loss']):
    
        df_var = lcurve 
        data = vars()[df_var]
        ax[index].plot(data, color='b' if lcurve=='accuracy' else 'r')
        ax[index].set_title( 'Training {}'.format(lcurve))
        if start_epoch >= FE_EPOCHS: 
            ax[index].plot([FE_EPOCHS, FE_EPOCHS], 
                 plt.ylim(), linestyle='--', label='Fine tuning 1')
        if start_epoch >= FE_EPOCHS+FT_EPOCHS: 
            ax[index].plot([FE_EPOCHS+FT_EPOCHS, FE_EPOCHS+FT_EPOCHS], 
                 plt.ylim(), linestyle='--', label='Fine Tuning 2')
        ax[index].legend(loc='lower left')
    
    plt.xlabel('epochs')
    fig.suptitle('Training curves',
                 y=1.1, fontsize=16)
    fig.tight_layout()
    

In [ ]:
#Callbacks
class LRFinder(Callback):
    
    '''
    The Learning Rate range test: a callback for finding the optimal learning rate range  
    This function will 
    
    # Usage
        ```
            lr_finder = LRFinder(min_lr=1e-5, 
                                 max_lr=1e-2, 
                                 steps_per_epoch=np.ceil(data_size/batch_size),  
                                 epochs=3
                                 beta=0.9)
            model.fit(X_train, Y_train, callbacks=[lr_finder])
            
            lr_finder.plot_loss()
        ```
    
    # Arguments
        min_lr: The lower bound of the learning rate  
        max_lr: The upper bound of the learning rate 
        steps_per_epoch: Number of iterations/mini-batches -- calculated as `np.ceil(data_size/batch_size)`. 
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient. 
        beta: the smoothing parameter. 0.99 ~ weighted over 100 previous values, 
                                       0.9 - 10 values.
        
    # Acknowledgements
        https://raw.githubusercontent.com/Meena-Mani/IDC_breast_cancer/master/lrate_callback.py
        Original paper: https://arxiv.org/abs/1506.01186

    '''
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None, beta=0.9):
        super().__init__()
        
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        self.beta = beta
        
    def clr(self):
        '''Calculate the learning rate.'''
        #use log fn to sample very small values
        x = np.log(1 + self.iteration / self.total_iterations) #use log fn to sample very small values
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        tf.keras.backend.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''For every iteration, record batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        tf.keras.backend.set_value(self.model.optimizer.lr, self.clr())
 

    def smooth_fn(self, y):
        '''Helper function to smooth input over a weighted average.'''
        n = len(self.history['iterations'])
        beta_c = 1 - self.beta
        ewa = np.zeros(n)
        ewa_corrected = np.zeros(n)
        ewa_corrected[0] = ewa[0] = y[0]
        for i in range (1,n):
            ewa[i] = self.beta*ewa[i-1] + beta_c*y[i] 
            ewa_corrected[i] = ewa[i] / (1 - self.beta**n)
        return ewa_corrected

    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.figure(figsize=(10,6))
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('LR')
        plt.title("Learning rate")
        
    def plot_loss(self):
        '''Plot the loss versus the learning rate'''
        plt.figure(figsize=(10,6))
        smoothed_loss = self.smooth_fn(self.history['loss'])
        plt.plot(self.history['lr'][1::10], smoothed_loss[1::10])
        plt.xscale('log')
        plt.xlabel('LR (log scale)')
        plt.ylabel('Loss')
        plt.title("Loss vs Learning Rate")


In [ ]:
# Callbacks


def lrfn(epoch, lr):
    
    if epoch == 1:
        lr = lr
    elif epoch<=3:
        lr = lr**1.1
    else :
        lr = lr / 2

    return lr


lr_scheduler = LearningRateScheduler(lrfn,verbose=1) 


checkpoint = ModelCheckpoint(filepath='/kaggle/working/best_weights.hdf5', 
                             save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

Transfer Learning step 1: Feature Extraction
EfficientNet considerations:
We use the EfficientNet B3 model since it is a mobile-sized architecture with 11M trainable parameters with a higher accuracy than models in this class.

Some strategies used for feature extraction:

small batch size for faster convergence to a good solution (batch size = 16)
larger learning rate (we use the default Adam lr: lr = 0.001)
no data augmentation
do not rebalance the data with oversampling; use class weights

In [ ]:
df_train = get_train_set(rebalance=0)

train_data = get_train_data(augmentation=0)

class_weights = get_class_weights()

In [ ]:
df_test = get_test_set()

In [ ]:
plot_dataset_distribution(df_train, df_test)

# ****The model!!****

In [ ]:
# Create the base model from the pre-trained model Resnet50
#base_model = ResNet50(input_shape=IMG_SHAPE,
                          #      include_top=False, 
                           #     weights='imagenet',
                           #    )

In [ ]:
img_in = Input(IMG_SHAPE) 
base_model = MobileNet(include_top= False , # remove  the 3 fully-connected layers at the top of the network
                weights='imagenet', # pre train weight
                input_shape= IMG_SHAPE
                     ) 

In [ ]:
#base_model.summary()

In [ ]:
base_model.trainable = False
base_model.summary()

In [ ]:
#print(IMG_SHAPE)

In [ ]:

model = tf.keras.Sequential([
  base_model,
  layers.GlobalAveragePooling2D(),
  layers.Dense(NUM_CLASSES, activation='softmax', name="predictions"),
])


In [ ]:
model.summary()
plot_model(model)

In [ ]:
OPTIMIZER = Adam(lr=0.001)
model.compile(optimizer=OPTIMIZER, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True,expand_nested=False)

# Find learning rate

In [ ]:
# function to find/plot the learning rate using the LRFinder callback

def find_lr(MIN_LR=1e-5, MAX_LR=1e-2, STEPS=train_data.samples // BATCH_SIZE, 
            EPOCHS=3, CLASS_WEIGHTS={0: 1.0, 1: 3.3, 2: 4.3, 3: 1.4}, 
            beta = 0.99):
    
    lr_finder = LRFinder(min_lr=MIN_LR, 
                         max_lr=MAX_LR, 
                         steps_per_epoch=STEPS, 
                         epochs=EPOCHS,
                         beta = beta)
    
    history = model.fit(
        train_data,
        steps_per_epoch=train_data.samples // BATCH_SIZE,
        epochs=3,
        #workers=3,
        callbacks=[lr_finder],
        class_weight=CLASS_WEIGHTS)
    
    lr_finder.plot_loss()
    lr_finder.plot_lr()

In [ ]:
find_lr(CLASS_WEIGHTS=class_weights)

A learning rate of 0.001 or 0.002 will give the best performance. The default for the Adam optimizer is 0.001 so we will use that value. This choice is supported by experiments we did using LR = 0.001, 0.002 and 0.01. As indicated by the LR curve above, 0.01 is probably too large and the model had problems converging. 0.002 was also suboptimal.

> **Feature Extraction**

In [ ]:
OPTIMIZER = Adam(lr=0.001)
model.compile(optimizer=OPTIMIZER, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [ ]:
history_FE = model.fit(
           train_data, 
           steps_per_epoch= train_data.samples // BATCH_SIZE, 
           epochs=FE_EPOCHS,
           class_weight=class_weights, 
           #workers=4,
           #validation_data=valid_data, 
           #validation_steps=valid_data.samples // BATCH_SIZE,
           callbacks=[checkpoint])

model.save_weights(FE_FILE)

In [ ]:
# plot training curves
    
accuracy = history_FE.history['accuracy']
loss = history_FE.history['loss']
display_training_curves(accuracy, loss, start_epoch=0)

****Transfer Learning step 2: Fine-tuning****

This is the second stage of transfer learning. We unfreeze some fraction of the layers and fit the model using smaller learning rate (Adam lr = 0.0001).

We address the data imbalance by:

rebalancing the training dataset (oversampling with copies of DME and DRUSEN)
augmenting the data
using class weights

In [ ]:
df_train = get_train_set(rebalance=1)
train_data = get_train_data(augmentation=1)
    
class_weights = get_class_weights()
    
plot_dataset_distribution(df_train, df_test)

In [ ]:
    
print("Number of layers in the base model: ", len(base_model.layers))
# plot_model(base_model)


base_model.trainable = True

# freezing all layers except the batch normalization layers
#for layer in base_model.layers:
 #   if isinstance(layer, BatchNormalization):
  #     layer.trainable = True
   # else:
    #    layer.trainable = False 

#set_trainable = False
for layer in base_model.layers:
    #if layer.name == 'block7a_se_excite':
     #   set_trainable = True
    #if set_trainable:
    if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
    else:
        layer.trainable = False

model.summary()

In [ ]:
OPTIMIZER = Adam(lr=1e-4)
model.compile(optimizer=OPTIMIZER, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
history_FT_1 = model.fit(
            train_data, 
            steps_per_epoch = train_data.samples // BATCH_SIZE, 
            epochs = FT_EPOCHS,
            class_weight = class_weights, 
            #workers = 4,
            #validation_data=valid_data, 
            #validation_steps=valid_data.samples // BATCH_SIZE,
            callbacks=[checkpoint, lr_scheduler])
    
model.save_weights(FT_FILE)


The training dataset is now more balanced (but not completely equal).

In [ ]:
# plot FE and FT learning curves 

accuracy += history_FT_1.history['accuracy']
loss += history_FT_1.history['loss']
display_training_curves(accuracy, loss, start_epoch=FE_EPOCHS)


****Test Performance****

In [ ]:
# for test resultes only
# model.load_weights(MODEL_WEIGHTS_PATH+FT_FILE)

In [ ]:
def plot_confusion_matrix(cm, 
                          classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Purples):
   

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy  
    
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)
    
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'
           .format(accuracy, misclass),fontsize=15)
    plt.ylabel('True label', fontsize=15)
    plt.title(title, fontsize=22);
    plt.colorbar()

    thresh = cm.max() / 2.0
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

In [ ]:
#df_test = get_test_set()
test_data = get_test_data()

In [ ]:
y_prob = model.predict(test_data, steps=len(test_data), verbose=1)
y_pred = np.argmax(y_prob,axis=1)
y_true = df_test['labels'].astype('int64').to_numpy()

In [ ]:
# print results

print(classification_report(y_true, y_pred))
print('The accuracy is {}'.format(accuracy_score(y_true, y_pred)))


In [ ]:
# confusion matrix
    
cm = confusion_matrix(y_true,y_pred)
classes = np.array(CLASSES)
title = 'Confusion matrix of results'
plot_confusion_matrix(cm, classes, title)

In [ ]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
print("Recall/TPR = {}".format(TPR))

# Specificity or true negative rate
TNR = TN/(TN+FP) 
print("Specificity/TNR = {}".format(TNR))

# Precision or positive predictive value
PPV = TP/(TP+FP)
print("Precision/PPV = {}".format(PPV))

# Negative predictive value
NPV = TN/(TN+FN)
print("Negative Predict Value = {}".format(NPV))

# Fall out or false positive rate
FPR = FP/(FP+TN)
print("False Positive Rate = {}".format(FPR))

# False negative rate
FNR = FN/(TP+FN)
print("False Negative Rate = {}".format(FNR))

# False discovery rate
FDR = FP/(TP+FP)
print("False discovery rate = {}".format(FDR))

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print("Overall Accuracy = {}".format(ACC))